# Fifth Project: Code Generation

Once semantic analysis are done, we can walk through the decorated AST to generate a linear
N-address code, analogously to [LLVM IR](https://llvm.org/docs/LangRef.html). We call this
intermediate machine code as uCIR. So, in this fifth project, you will turn the AST into
uCIR. uCIR uses a Single Static Assignment (SSA), and can promote stack allocated scalars
to virtual registers and remove the load and store operations, allowing better
optimizations since values propagate directly to their use sites.  The main thing that
distinguishes SSA from a conventional three-address code is that all assignments in SSA
are for distinguished name variables.

Once your compiler emits intermediate code, you should be able to run the code using a
simple interpreter provided for this purpose.  This can be useful for testing, and other 
tasks involving the generated code.

## Intermediate Representation

At this stage of the project, you are going to turn the decorated AST into an intermediate
machine code named uCIR based on Single Static Assignment (SSA). There are a few important
parts you will need to make this work. Please read carefully before beginning:

### Single Static Assignment
The first problem is how to decompose complex expressions into
something that can be handled more simply.  One way to do this is
to decompose all expressions into a sequence of simple assignments
involving binary or unary operations.  

As an example, suppose you had a mathematical expression like this:
```
        2 + 3 * 4 - 5
```
Here is one possible way to decompose the expression into simple
operations:
```
        %1 = 2
        %2 = 3
        %3 = 4
        %4 = %2 * %3
        %5 = %1 + %4
        %6 = 5
        %7 = %5 - %6
```
In this code, the **%n** variables are simply temporaries used while
carrying out the calculation.  A critical feature of SSA is that such
temporary variables are only assigned once (single assignment) and
never reused.  Thus, if you were to evaluate another expression, you
would simply keep incrementing the numbers. For example, if you were
to evaluate **10 + 20 + 30**, you would have code like this:
```
        %8 = 10
        %9 = 20
        %10 = %8 + %9
        %11 = 30
        %12 = %10 + %11
```
SSA is meant to mimic the low-level instructions one might carry out 
on a CPU.  Another benefit of SSA is that it is very easy to encode and
manipulate using simple data structures such as tuples. For example,
you could encode the above sequence of operations as a list like this:

```python
       [ 
         ('store', 2, '%1'),
         ('store', 3, '%2'),
         ('store', 4, '%3'),
         ('mul', '%2', '%3', '%4'),
         ('add', '%1', '%4', '%5'),
         ('store', 5, '%6'),
         ('sub', '%5', '%6', '%7'),
       ]
```

### Dealing with Variables
In your program, you are probably going to have some variables that get
used and assigned different values.  For example:
```
       a = 10 + 20;
       b = 2 * a;
       a = a + 1;
```
In "pure SSA", all of your variables would actually be versioned just
like temporaries in the expressions above.  For example, you would
emit code like this:
```
       %1 = 10
       %2 = 20
       a_1 = %1 + %2
       %4 = 2
       b_1 = %4 * a_1
       %5 = 1 
       a_2 = a_1 + %5
       ...
```
To avoid this, we're going to treat declared variables as memory locations and access
them using load/store instructions.  For example:
```
       %1 = 10
       %2 = 20
       %3 = %1 + %2
       store(%3, "a")
       %4 = 2
       %5 = load("a")
       %6 = %4 * %5
       store(%6,"b")
       %7 = load("a")
       %8 = 1
       %9 = %7 + %8
       store(%9, "a")
```

### A Word About Types
At a low-level, CPUs can only operate a few different kinds of 
data such as ints and floats.  Because the semantics of the
low-level types might vary slightly, you'll need to take 
some steps to handle them separately.

In our intermediate code, we're simply going to tag temporary variable
names and instructions with an associated type low-level type.  For
example:

      2 + 3 * 4          (ints)
      2.0 + 3.0 * 4.0    (floats)

The generated intermediate code might look like this:

```python
      ('literal_int', 2, '%1')
      ('literal_int', 3, '%2')
      ('literal_int', 4, '%3')
      ('mul_int', '%2', '%3', '%4')
      ('add_int', '%1', '%4', '%5')

      ('literal_float', 2.0, '%6')
      ('literal_float', 3.0, '%7')
      ('literal_float', 4.0, '%8')
      ('mul_float', '%7', '%8', '%9')
      ('add_float', '%6', '%9', '%10')
```

### Your Task
Your task is as follows: Write a AST Visitor() class that takes an
uC program and flattens it to a single sequence of SSA code instructions
represented as tuples of the form 
```python
       (operation, operands, ..., destination)
```
Your SSA code should only contain the following operators:

#### Variables & Values:
```python
      ('alloc_type', varname)              # Allocate on stack (ref by register) a variable of a given type.
      ('global_type', varname, value)      # Allocate on heap a global var of a given type. value is optional.
      ('load_type', varname, target)       # Load the value of a variable (stack/heap) into target (register).
      ('store_type', source, target)       # Store the source/register into target/varname.
      ('literal_type', value, target)      # Load a literal value into target.
      ('elem_type', source, index, target) # Load into target the address of source (array) indexed by index.
      ('get_type', source, target)         # Store into target the address of source (used for pointers).
```

#### Binary Operations:
```python
       ('add_type', left, right, target)   # target = left + right
       ('sub_type', left, right, target)   # target = left - right
       ('mul_type', left, right, target)   # target = left * right
       ('div_type', left, right, target)   # target = left / right  (integer truncation)
       ('mod_type', left, right, target)   # target = left % rigth
```

#### Cast Operations:
```python
       ('fptosi', fvalue)                   # (int)fvalue == cast float to int 
       ('sitofp', ivalue)                   # (float)ivalue == cast int to float
```

#### Relational/Equality/Logical:
```
       ('oper_type', left, right, target)   # target = left `oper` rigth, where `oper` is:
                                                  lt, le, ge, gt, eq, ne, and, or, not
```

#### Labels & Branches:
```python
       ('label:', )                                       # Label definition
       ('jump', target)                                   # Jump to a target label
       ('cbranch, expr_test, true_target, false_target)   # Conditional Branch
```

#### Functions & Builtins:
```python
       ('define_type', source, args)    # Function definition. Source=function label, args=list of pairs
                                        # (type, name) of formal arguments. 
       ('call_type', source, target)    # Call a function. target is an optional return value
       ('return_type', target)          # Return from function. target is an optional return value
       ('param_type', source)           # source is an actual parameter
       ('read_type', source)            # Read value to source
       ('print_type',source)            # Print value of source
```

## Basic Block Creation

During the construction of the sequence of instructions in its intermediate representation
for the program, we will also build the basic blocks. A basic block is a sequence of
instructions where the control flow enters only at the beginning of the block and exits
at the end of the block, without the possibility of deviation to any other part of the
program. The basic blocks make up the nodes of a control flow graph (CFG), a structure
that will be important for performing code optimizations.

When building the basic blocks, you may note that the first statement of a basic block
is always a label and the last statement is a jump statement (conditional or unconditional). Function calls should be treated as straight-line IR nodes (i.e., they are not treated as branches; their successor is the instruction immediately after the call). Return nodes do not have any successors.

  - Unconditional jumps (like at the end of for loops) have only one successor:
the target of the jump statement. When you see an unconditional jump, add the target of
the jump statement as a successor of the jump, and the jump statement as a predecessor of
the target.

  - Conditional jumps have two successors: the ```fall_through``` target, which can be
  a successor in the linked list, and the ```taken``` target. Add the branch as a
  predecessor of the taken target, and the taken target as an additional successor of
  the branch.

The following classes `Block`, `BasicBlock` and `ConditionBlock` are provided to help you implement blocks for the intermediate representation:

In [ ]:
class Block(object):
    def __init__(self, label):
        self.label = label       # Label that identifies the block
        self.instructions = []   # Instructions in the block
        self.predecessors = []   # List of predecessors
        self.next_block = None   # Link to the next block

    def append(self,instr):
        self.instructions.append(instr)

    def __iter__(self):
        return iter(self.instructions)

class BasicBlock(Block):
    '''
    Class for a simple basic block.  Control flow unconditionally
    flows to the next block.
    '''
    def __init__(self, label):
        super(BasicBlock, self).__init__(label)
        self.branch = None  # Not necessary the same as next_block in the linked list

class ConditionBlock(Block):
    """
    Class for a block representing an conditional statement.
    There are two branches to handle each possibility.
    """
    def __init__(self, label):
        super(ConditionBlock, self).__init__(label)
        self.taken = None
        self.fall_through = None

## uCIR Examples
Below you find a simple example of the intermediate representation (IR) for the given uC
program. More examples are provided in the [uCIR_Examples](./uCIR_Examples.ipynb) notebook.

uC:
```c
int n = 10;

int foo(int a, int b) {
    return n * (a + b);
}

int main() {
    int c = 2, d = 3;
    int e = foo(c, d);
    return 0;
}
```
IR:
```python
('global_int', '@n', 10)
('define_int', '@foo', [('int', '%1'), ('int', '%2')])
# function arguments: the value for "a" is passsed in register %1, for "b" in register %2
# & register %3 is reserved to hold the return value
('entry:',)
('alloc_int', '%3')
('alloc_int', '%a')
('alloc_int', '%b')
('store_int', '%1', '%a')
('store_int', '%2', '%b')
('load_int', '%a', '%4')
('load_int', '%b', '%5')
('add_int', '%4', '%5', '%6')
('load_int', '@n', '%7')
('mul_int', '%7', '%6', '%8')
('store_int', '%8', '%3')
('jump', '%exit')
('exit:',)
('load_int', '%3', '%9')
('return_int', '%9')

('define_int', '@main', [])
# the main in uC has no arguments, only register %1 is reserved for return
('entry:',)
('alloc_int', '%1')
('alloc_int', '%c')
('alloc_int', '%d')
('alloc_int', '%e')
('literal_int', 2, '%2')
('store_int', '%2', '%c')
('literal_int', 3, '%3')
('store_int', '%3', '%d')
('load_int', '%c', '%4')
('load_int', '%d', '%5')
('param_int', '%4')
('param_int', '%5')
('call_int', 'foo', '%6')
('store_int', '%6', '%e')
('literal_int', 0, '%7')
('store_int', '%7', '%1')
('jump', '%exit')
('exit:',)
('load_int', '%1', '%8')
('return_int', '%8')
```

### Prettyprint
We will apply a stylistic formating to the uCIR intermediary representation to facilitate the
content so that you can view, read and understand it more easily. For this, you can use the
following function that can be adapted to your style.

In [1]:
def format_instruction(t):
    # Auxiliary method to pretty print the instructions 
    # t is the tuple that contains one instruction
    operand = t[0].split('_')
    op = operand[0]
    ty = operand[1] if len(operand) > 1 else None
    if len(operand) >= 3:
        for _qual in operand[2:]:
            if _qual == '*':
                ty += '*'
            else:
                ty += f" [{_qual}]"
    if len(t) > 1:
        if op == "define":
            return f"\n{op} {ty} {t[1]} " + ', '.join(list(' '.join(el) for el in t[2]))
        else:
            _str = "" if op == "global" else "  "
            if op == 'jump':
                _str += f"{op} label {t[1]}"
            elif op == 'cbranch':
                _str += f"{op} {t[1]} label {t[2]} label {t[3]}"
            elif op == "global":
                if ty.startswith('string'):
                    _str += f"{t[1]} = {op} {ty} \'{t[2]}\'"
                else:
                    _str += f"{t[1]} = {op} {ty} {t[2]}"
            elif op == "return" or op == "print":
                _str += f"{op} {ty} {t[1]}"
            elif op == "sitofp" or op == "fptosi":
                _str += f"{t[2]} = {op} {t[1]}"                
            elif op == "store" or op == "param":
                _str += f"{op} {ty} "
                for _el in t[1:]:
                    _str += f"{_el} "
            else:
                _str += f"{t[-1]} = {op} {ty} "
                for _el in t[1:-1]:
                    _str += f"{_el} "
            return _str
    elif ty == 'void':
        return f"  {op}"
    else:
        return f"{op}"

Next, the previous example is presented in this format. All the examples 
to follow in this and other notebooks will follow this formatting style.

```
@n = global int 10

define int @foo (int %1, int %2)
entry:
  %3 = alloc int 
  %a = alloc int 
  %b = alloc int 
  store int %1 %a 
  store int %2 %b 
  %4 = load int %a 
  %5 = load int %b 
  %6 = add int %4 %5 
  %7 = load int @n 
  %8 = mul int %7 %6 
  store int %8 %3 
  jump label %exit
exit:
  %9 = load int %3 
  return int %9

define int @main ()
entry:
  %1 = alloc int 
  %c = alloc int 
  %d = alloc int 
  %e = alloc int 
  %2 = literal int 2 
  store int %2 %c 
  %3 = literal int 3 
  store int %3 %d 
  %4 = load int %c 
  %5 = load int %d 
  param int %4
  param int %5
  %6 = call int @foo 
  store int %6 %e 
  %7 = literal int 0 
  store int %7 %1 
  jump label %exit
exit:
  %8 = load int %1 
  return int %8
```

## A note about arrays
The dimensions of an Array in the uC are known at compile time. Then, the type described
in the allocation must express the dimension of the same. The initializer_list are always
allocated in the heap, either directly in the declaration of the variable, if it is
global, or by defining a new temporary, based on the name of the local variable. Examples:

uC:
```c
int x[] = {1, 2, 3};
void main(){}
```
IR:
```
@x = global int[3] [1, 2, 3] 

define_void @main ()
entry:
exit:
  return
```

uC:
```c
int x[2][2];
void main(){
  int y[] = {1, 2, 3};
}
```
IR:
```
@x = global int[2][2]
@.const_y.0 = global int[3] [1, 2, 3]

define void @main ()
entry:
  %y = alloc int[3] 
  store int[3] @.const_y.0 %y 
exit:
  return
```

## A note about Pointers
The allocation and operations with pointers in UC follow the same structure used for arrays.
The exception is that reading the referenced value requires two instructions. See the
following example:

uC:
```c
int main () {
    int x, y;
    int *r = &x;
    *r = y;
    x = *r;
    return 1;
}
```
IR:
```
define int @main ()
entry:
  %1 = alloc int 
  %x = alloc int 
  %y = alloc int 
  %r = alloc int* 
  %r = get int* %x 
  %2 = load int %y 
  store int* %2 %r 
  %3 = load int* %r 
  store int %3 %x 
  %4 = literal int 1 
  store int %4 %1 
  jump label %exit
exit:
  %5 = load int %1 
  return int %5
```

# Generating Code

Implement the following GenerateCode class so that it creates
a sequence of SSA instructions in the form of tuples.

For walking the AST, use the NodeVisitor class. An initial implementation of the code generation is provided in the code below.

In [ ]:
class GenerateCode(NodeVisitor):
    """
    Node visitor class that creates 3-address encoded instruction sequences
    with Basic Blocks & Control Flow Graph.
    """
    def __init__(self, viewcfg):
        self.viewcfg = viewcfg
        self.current_block = None

        # version dictionary for temporaries. We use the name as a Key
        self.fname = '_glob_'
        self.versions = {self.fname:0}

        # The generated code (list of tuples)
        # At the end of visit_program, we call each function definition to emit
        # the instructions inside basic blocks. The global instructions that
        # are stored in self.text are appended at beginning of the code
        self.code = []
        
        self.text = []  # Used for global declarations & constants (list, strings)
        
        # TODO: Complete if needed.


    def new_temp(self):
        """
        Create a new temporary variable of a given scope (function name).
        """
        if self.fname not in self.versions:
            self.versions[self.fname] = 1
        name = "%" + "%d" % (self.versions[self.fname])
        self.versions[self.fname] += 1
        return name

    def new_text(self, typename):
        """
        Create a new literal constant on global section (text).
        """
        name = "@." + typename + "." + "%d" % (self.versions['_glob_'])
        self.versions['_glob_'] += 1
        return name

    # You must implement visit_Nodename methods for all of the other
    # AST nodes.  In your code, you will need to make instructions
    # and append them to the current block code list.
    #
    # A few sample methods follow. Do not hesitate to complete or change 
    # them if needed.

    def visit_Constant(self, node):
        if node.type.name == "string":
            _target = self.new_text('str')
            inst = ('global_string', _target, node.value)
            self.text.append(inst)
        else:
            # Create a new temporary variable name
            _target = self.new_temp()
            # Make the SSA opcode and append to list of generated instructions
            inst = ('literal_' + node.type.name, node.value, _target)
            self.current_block.append(inst)
        # Save the name of the temporary variable where the value was placed
        node.gen_location = _target

    def visit_BinaryOp(self, node):
        # Visit the left and right expressions
        self.visit(node.left)
        self.visit(node.right)
        
        # TODO: 
        # - Load the location containing the left expression
        # - Load the location containing the right expression

        # Make a new temporary for storing the result
        target = self.new_temp()

        # Create the opcode and append to list
        opcode = binary_ops[node.op] + "_" + node.left.type.name
        inst = (opcode, node.left.gen_location, node.right.gen_location, target)
        self.current_block.append(inst)

        # Store location of the result on the node
        node.gen_location = target

    def visit_Print(self, node):
        # Visit the expression
        self.visit(node.expr)

        # TODO: Load the location containing the expression

        # Create the opcode and append to list
        inst = ('print_' + node.expr.type.name, node.expr.gen_location)
        self.current_block.append(inst)
        
        # TODO: Handle the cases when node.expr is None or ExprList

    def visit_VarDecl(self, node):
        # Allocate on stack memory
        _varname = '%' + node.declname.name
        inst = ('alloc_' + node.type.name, _varname)
        self.current_block.append(inst)
        
        # Store optional init val
        _init = node.decl.init
        if _init is not None:
            self.visit(_init)
            inst = ('store_' + node.type.name, _init.gen_location, node.declname.gen_location)
            self.current_block.append(inst)
            
    # TODO: Complete.

### Program / Functions

1. Program (`visit_Program`)

Start by visiting all global declarations. Then, visit all the function definitions and emit the code stored inside basic blocks.

2. Function Definition (`visit_FuncDef`)

Initialize the necessary blocks to construct the CFG of the function. Visit the function declaration. Visit all the declarations within the function. After allocating all declarations, visit the arguments initialization. Visit the body of the function to generate its code. Finally, setup the return block correctly and generate the return statement (even for void function).

3. Parameter list (`visit_ParamList`)

Just visit all arguments.

### Declarations / Type

1. Global Declaration (`visit_GlobalDecl`)

Visit each global declaration that are not function declarations. Indeed, it is usually simpler to visit the function declaration when visiting the definition of the function to generate all code at once.

2. Declaration (`visit_Decl`)

Visit the type of the node.

3. Variable Declaration (`visit_VarDecl`)

Allocate the variable (global or local) with the correct initial value (if there is any).

4. Array Declaration (`visit_ArrayDecl`)

Visit the node type.

5. Function Declaration (`visit_FuncDecl`)

Generate the function definition (including function name, return type and arguments types). This is also a good time to generate the entry point for function, allocate a temporary for the return statement (if not a void function), and visit the arguments.

6. Declaration List (`visit_DeclList`)

Visit all of the declarations that appear inside for statement.

7. Type (`visit_Type`)

Do nothing: just `pass`.

### Statements

While moving along blocks, you need to update the reference to the current block and reference its predecessors.

1. If Statement (`visit_If`)

First, generate the evaluation of the condition (*visit it*). Create the required blocks and the branch for the condition. Move to the first block and generate the statement related to the *then*, create the branch to exit. In case, there is an *else* block, generate it in a similar way.

2. For Statement (`visit_For`)

First, generate the initialization of the For and creates all the blocks required. Then, generate the jump to the condition block and generate the condition and the correct conditional branch. Generate the body of the For followed by the jump to the increment block. Generate the increment and the correct jump.

3. While Statement (`visit_While`)

The generation of While is similar to For except that it does not require the part related to initialization and increment.

4. Compound Statement (`visit_Compound`)

Visit the list of block items (declarations or statements).

5. Assignement (`visit_Assignment`)

First, visit right side and load the value according to its type. Then, visit the left side and generate the code according to the assignment operator and the type of the expression (`ID` or `ArrayRef`).

6. Break Statement (`visit_Break`)

Generate a `jump` instruction to the current exit label.

7. Funcion Call (`visit_FuncCall`)

Start by generating the code for the arguments: for each one of them, visit the expression and generate a `param_type` instruction with its value. Then, allocate a temporary for the return value and generate the code to call the function.

8. Assert Statement (`visit_Assert`)

The assert is a conditional statement which generate code quite similar to the
If Block. If the expression is false, the program should issue an error message
(assertfail) and terminate. If the expression is true, the program proceeds to
the next sentence.

Visit the assert condition. Create the blocks for the condition and adust their predecessors. Generate the branch instruction and adjust the blocks to jump according to the condition. Generate the code for unsuccessful assert, generate the print instruction and the jump instruction to the return block, and  successful assert. 

9. Empty Statement (`visit_EmptyStatement`)

Do nothing, just `pass`.

10. Print Statement (`visit_Print`)

If the expression is empty, generate a `print_void` instruction. Otherwise, you need to visit each expression, load it if necessary and generate a print instruction for each one.

11. Read Statement (`visit_Read`)

For each name, you need to visit it, load it if necessary and generate a read instruction for each element.

12. Return Statement (`visit_Return`)

If there is an expression, you need to visit it, load it if necessary and store its value to the return location. Then generate a jump to the return block if needed. Do not forget to update the predecessor of the return block.

### Expressions

For each expression, create a new temporary variable, create an instruction to save the value of the expression in the variable and save the name of the temporary variable where the value was placed as an attribute of the related AST node (called `gen_location` in the sample code).

1. Constant (`visit_Constant`)

If the constant is of type string, create a new global that will contain the value. Otherwise just create a new temporary initialized with the value.

2. Identifier (`visit_ID`)

Get the name of the temporary (or register) where the value of the variable is
stored. This temporary (gen_location, in the sample code) was stored next to
the variable's declaration during its allocation. For this, you can consult
the symbol table or use the bind attribute that's link the identifier with
its declaration (usually made during semantic analysis).

2. Cast operation (`visit_Cast`)

Start by visiting the expression. Then, create the correct cast instruction (`fptosi` or `sitofp`) according to the target type (`to_type`).

3. Binary Operation (`visit_BinaryOp`)

Visit the left and right expressions to generate the code related to them. Load their value if they reference an array. Create a new instruction with the correct opcode and store its result in a new temporary variable.

4. Unary Operation (`visit_UnaryOp`)

The generation of unary operations are similar to binary operations except that they have only a single expression.

5. Expression List (`visit_ExprList`)

Do nothing, just `pass`: the Expression List must be treated in the scope that uses it.

6. Array Reference (`visit_ArrayRef`)

Start by visiting the subscript. Load the values of the index in a new temporary. If the array has multiple dimensions: you need to generate arithmetic instructions to compute the index of the element in the array.

7. Initialization List (`visit_InitList`)

Evaluate each element of the list and add its value to the node value (which is a list).

## Viewing the Control Flow Graph
Let's create a graphic object and assemble the graphic by adding the generated CFG nodes
and edges. For this we will use graphviz (```pip3 install graphviz```), a package that
facilitates the creation and rendering of graph descriptions in the DOT language of
Graphviz graphical drawing software.

In the code below, we use the option/viewing method to directly inspect the resulting
file in PDF format. The graphics can also be saved in files containing the generated
DOT source code.

In [ ]:
class CFG(object):

    def __init__(self, fname):
        self.fname = fname
        self.g = Digraph('g', filename=fname + '.gv', node_attr={'shape': 'record'})

    def visit_BasicBlock(self, block):
        # Get the label as node name
        _name = block.label
        if _name:
            # get the formatted instructions as node label
            _label = "{" + _name + ":\l\t"
            for _inst in block.instructions[1:]:
                _label += format_instruction(_inst) + "\l\t"
            _label += "}"
            self.g.node(_name, label=_label)
            if block.branch:
                self.g.edge(_name, block.branch.label)
        else:
            # Function definition. An empty block that connect to the Entry Block
            self.g.node(self.fname, label=None, _attributes={'shape': 'ellipse'})
            self.g.edge(self.fname, block.next_block.label)

    def visit_ConditionBlock(self, block):
        # Get the label as node name
        _name = block.label
        # get the formatted instructions as node label
        _label = "{" + _name + ":\l\t"
        for _inst in block.instructions[1:]:
            _label += format_instruction(_inst) + "\l\t"
        _label +="|{<f0>T|<f1>F}}"
        self.g.node(_name, label=_label)
        self.g.edge(_name + ":f0", block.taken.label)
        self.g.edge(_name + ":f1", block.fall_through.label)

    def view(self, block):
        while isinstance(block, Block):
            name = "visit_%s" % type(block).__name__
            if hasattr(self, name):
                getattr(self, name)(block)
            block = block.next_block
        # You can use the next stmt to see the dot file
        # print(self.g.source)
        self.g.view()

#
# At the end of visit_Program method in the Code Generator Class you call CFG view method
#

    def visit_Program(self):
        # ...
        if self.viewcfg:  # evaluate to True if -cfg flag is present in command line
            for _decl in node.gdecls:
                if isinstance(_decl, FuncDef):
                    dot = CFG(_decl.decl.name.name)
                    dot.view(_decl.cfg)  # _decl.cfg contains the CFG for the function

### Example:
```
int checkPrime(int n) {
    int i, isPrime = 1;
    for (i = 2; i <= n/2; ++i) {
        if (n % i == 0) {
            isPrime = 0;
            break;
        }
    }
    return isPrime;
}
```

<img src=".\doc\checkPrime.gv.png" alt="Drawing" style="width: 420px;"/>

# Run the uCIR in the Interpreter

Once you've got your compiler emitting intermediate code, you should be able to runs
the code in the [interpreter](./src/uc_interpreter.py). This can be useful for
testing, and other tasks involving the generated code.

You can think the Interpreter as a kind of stack machine, which means
that most instructions take their operands from the stack, and place
results back on the stack.

It defines a memory model that consists of a program memory (the code),
a dictionary to hold references (indexes) to vars, labels & registers
in the memory (M). All the data areas of M are divided into cells,
and each cell can hold a single value. The actual size of a cell is
large enough to hold single values (int, char, bool and ref) or any
element of string (chars) and arrays. For simplicity, we use a separate
dictionary to hold the indexes of globals vars and constants.
These vars & constants will be previously stored at begining of the
memory by the interpreter before start running the program.

The Interpreter uses a program counter “pc” to fetches instructions
from the code. In this model, the M stack does not act as a function
stack for holding function linkage information but only data. It
 uses auxiliares stack and dictionaries to holding these informations.

 You can run the Interpreter in debug mode passing the '-d' flag to
 the compiler (see the [main module](./src/uc.py)). In this case, the
 interpreter issues the following text describing the debugging options:

 ```
Interpreter running in debug mode:

          s, step: run in step mode;
          g, go <pc>:  goto the program counter;
          l, list {<start> <end>}? : List the ir code;
          e, ex {<vars>}+ : Examine the variables;
          a, assign <var> <type> <value>: Assign the value of given type to var;
          v, view : show he current line of execution;
          r, run : run the program in normal mode;
          q, quit : quit the program;
          h, help: print this text.
```
In the debug mode you can run your program step by step, examine the content
of variables, assign a value to simple variables or elements of an array. In
the case of matrices of two dimensions, you must access an element in a
linearized way. You can also list the intermediate code and run the program
to a specific point in it.
The comments and docstrings in the Interpreter gives you more details.
